# TF-IDF

In [1]:
import pandas as pd
import numpy as np

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [2]:
date_str = '20250818'

In [3]:
# 파일 불러오기
raw_data_gen = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_general_{date_str}.csv")
raw_data_i = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_i_{date_str}.csv")
raw_data_g = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_g_{date_str}.csv")

# 하나의 파일로 유니온
raw_data = pd.concat([raw_data_gen, raw_data_i, raw_data_g], ignore_index=True)
print(raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

(349, 4)


,url,title,text,publish_date,text_clean,title_clean
0,https://www.hankyung.com/article/2025081847244,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다 회담은 뉴욕 증시가 마감할 무렵 ...,한국경제
2,https://www.hankyung.com/article/2025081847727,"美 트럼프 무역장벽에 '도미노', 韓 직격탄…규제 철강 집중 [영상]",영상 모듈 닫기\n\n/사진=뉴스1\n\n미국이 수입 철강 제품에 50%의 품목 관...,2025-08-18 06:46:02+09:00,영상 모듈 닫기 사진 뉴스 미국이 수입 철강 제품에 의 품목 관세를 부...,트럼프 무역장벽에 도미노 직격탄 규제 철강 집중 영상
3,https://www.hankyung.com/article/2025081847767,"""10년 전 격전 재연한다""…'애플왕국' 맹공하는 삼성전자","갤럭시 폴드7 등 선보이며\n\n미국 내 점유율 크게 올려\n\n\n\nCNBC ""...",2025-08-18 06:41:11+09:00,갤럭시 폴드 등 선보이며 미국 내 점유율 크게 올려 CNBC 애플도 폴더...,년 전 격전 재연한다 애플왕국 맹공하는 삼성전자


In [33]:
# 불용어제거 -> 형태소 조사 삭제 -> tf-idf -> 가장 중요한 10개 단어 뽑아내기

In [4]:
# 불용어 불러오기 (줄바꿈 기준 분리)
with open("/Users/leesangwon/Documents/ThemeStock_file/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 조사 제거 → 불용어 제거 순차 적용
df['title_clean'] = df['title_clean'].apply(remove_josa)  # 조사 제거
df['title_clean'] = df['title_clean'].apply(remove_stopwords)  # 불용어 제거

df['text_clean'] = df['text_clean'].apply(remove_josa)  # 조사 제거
df['text_clean'] = df['text_clean'].apply(remove_stopwords)  # 불용어 제거

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1) 코퍼스 준비 (NaN 방지)
corpus = df['text_clean'].fillna('')

# 2) 벡터라이저 설정
# - tokenizer=lambda s: s.split(): 이미 공백으로 토큰화된 텍스트를 그대로 사용
# - lowercase=False: 한글/대소문 유지
# - ngram_range=(1,1): 단어 단위(필요 시 (1,2) 등으로 확장)
# - min_df, max_df, max_features는 데이터 규모에 맞게 조절
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    token_pattern=None,
    lowercase=False,
    ngram_range=(1,1),
    min_df=2,           # 2개 미만 문서에만 나온 단어 제거
    max_df=0.7,         # 70% 넘는 문서에 등장하는 흔한 단어 제거
    max_features=8000   # 상위 8천개 단어만 사용 (메모리 절약용)
)

# 3) 학습 및 변환 (희소행렬)
X = vectorizer.fit_transform(corpus)   # shape: (문서수, 단어수)
terms = vectorizer.get_feature_names_out()

# 4) (옵션) DataFrame으로 시각화 — 메모리 주의
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=terms)

# 5) 각 문서별 TF-IDF 상위 키워드 뽑기 함수
def top_tfidf_terms(row_index: int, topk: int = 10):
    row = X.getrow(row_index).toarray().ravel()
    if not np.any(row):
        return []
    idx = np.argsort(row)[::-1][:topk]
    return list(zip(terms[idx], row[idx]))

# 6) 모든 문서에 대해 상위 키워드 뽑아 df에 붙이기 (리스트→문자열)
topk = 10
df['tfidf_top_terms'] = [
    ', '.join([f'{t}:{w:.3f}' for t, w in top_tfidf_terms(i, topk)]) 
    for i in range(X.shape[0])
]

In [6]:
df.head()

,url,title,text,publish_date,text_clean,title_clean,tfidf_top_terms
0,https://www.hankyung.com/article/2025081847244,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령 푸틴 러시아 대통령 만났습니다 회담 뉴욕 증시 마감 관망 나타났습니다...,한국,"관세:0.429, 인플레이션:0.241, 금리:0.240, 물가:0.227, 파월:..."
2,https://www.hankyung.com/article/2025081847727,"美 트럼프 무역장벽에 '도미노', 韓 직격탄…규제 철강 집중 [영상]",영상 모듈 닫기\n\n/사진=뉴스1\n\n미국이 수입 철강 제품에 50%의 품목 관...,2025-08-18 06:46:02+09:00,영상 모듈 닫기 뉴스 미국 수입 철강 제품 품목 관세 부과 보호무역 주의 강화한 나...,트럼프 무역 장벽 미노 직격탄 규제 철강 집중 영상,"국산:0.377, 규제:0.376, 수입:0.353, 철강:0.275, 무역:0.2..."
3,https://www.hankyung.com/article/2025081847767,"""10년 전 격전 재연한다""…'애플왕국' 맹공하는 삼성전자","갤럭시 폴드7 등 선보이며\n\n미국 내 점유율 크게 올려\n\n\n\nCNBC ""...",2025-08-18 06:41:11+09:00,갤럭시 폴드 선보이며 미국 점유 크게 올려 CNBC 애플 폴 더블 내놓으며 대응 나...,격전 재연 애플 왕국 맹공하 삼성,"애플:0.402, 더블:0.349, 스마트폰:0.349, 폴:0.349, 아이폰:0..."
5,https://www.hankyung.com/article/2025081847887,밥 사먹을 돈도 없는데 '술이 웬말이냐'…비명 터졌다,"KCD 2025년 2분기 소상공인 동향\n\n술집, 1년 전 대비 매출 9.2% 감...",2025-08-18 06:55:53+09:00,KCD 소상 공인 동향 술집 매출 감소 외식 업종 감소 폭 커 한국 신용 데이터 조...,밥 사먹을 돈 없는데 웬 비명 터졌다,"매출:0.350, 소상:0.343, 공인:0.304, 업종:0.302, 소비:0.2..."
11,https://www.hankyung.com/article/2025081847987,"우크라 종전 시나리오 나오나…美 ""우크라 안전보장 할 수도"" [영상]",트럼프 만나러 몰려가는 유럽 정상\n\n영상 모듈 닫기\n\n왼쪽부터 도널드 트럼프...,2025-08-18 07:15:25+09:00,트럼프 만나러 몰려가는 유럽 정상 영상 모듈 닫기 왼쪽 도널드 트럼프 미국 대통령 ...,크라 종전 시나리오 나오나 크라 안전보장 수도 영상,"우크라이나:0.530, 트럼프:0.294, 보장:0.265, 대통령:0.234, 안..."


In [7]:
from collections import Counter
import re

# 1. 전체 키워드 모으기 (점수 제거)
all_terms = []
for row in df['tfidf_top_terms']:
    if not isinstance(row, str):
        continue
    # "단어:점수" → 단어만 추출
    terms = [re.split(r':', t)[0] for t in row.split(', ')]
    all_terms.extend(terms)

# 2. 빈도수 계산
term_freq = Counter(all_terms)

# 3. DataFrame으로 변환
term_freq_df = pd.DataFrame(term_freq.items(), columns=['term', 'count'])
term_freq_df = term_freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

term_freq_df.head(10)

,term,count
0,대통령,16
1,,15
2,미국,13
3,관세,10
4,AI,10
5,영업,10
6,한국,10
7,거래,8
8,중국,8
9,매출,8


In [8]:
term_freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/TFIDF_{date_str}.csv", index=False)